# Recommender System

## 1. Introduction

In this primer, we will investigate a simple approach called *collaborative filtering* that only uses user-item information, for example

$$X = \begin{pmatrix}
    ? & 3 \\
    ? & 1 \\
    2 & 3 \\
\end{pmatrix}.$$

In this case, the ? entries are ratings that we do not have, either because the user has not bought a certain item or has bought it but did not leave a rating. The task of collaborative filtering is to "fill in" the missing values (i.e., the predicted user ratings) based on the existing ratings. Once we have these predictions, we can recommend the item with the highest predicted rating for each user.

There are three different approaches to the collaborative filtering problem:
1. **User-user approach**: estimate a user’s rating of an item by finding "similar" users and then looking at their ratings for that item.
2. **Item-item approach**: estimate a user’s rating of an item by finding similar items and then looking at that user's rating of these similar items.
3. **Matrix factorization**: construct two low-rank matrices that approximate the observed entries of $X$.

We will take a look at each of these representations below. From now we will assume that **all the ratings are positive, so missing entries $X_{ij}$ will be filled with 0**. In practice, assuming your data is stored in a Pandas dataframe, this effect can be achieved through calling `df.fillna(0)`. You can then convert a dataframe to a Numpy matrix with `df.to_numpy()`.

## 2. User-user approach

### 2.1 Definition

Assume $X_{ij}$ is a missing entry in our user-rating matrix. In the user-user approach we can, as a first step, think of estimating $X_{ij}$ as
$$\hat X_{ij} = \frac{  \sum_{k : X_{kj} \ne 0} w_{ik}X_{kj}}{  \sum_{k : X_{kj} \ne 0} w_{ik}},$$
where $w_{ik}$ is the similarity weight between user $i$ and user $k$.

In other words, to predict user $i$'s rating for item $j$, we look at every other user $k$ who has rated item $j$, multiply their rating ($X_{kj}$) with the weighted similarity between $k$ and $i$ (i.e., $w_{ik}$), then add these products together. Intuitively, if user $k$, who is very "similar" to user $i$, has rated item $j$, then we expect that $X_{kj}$ will be close to the unknown $X_{ij}$ and therefore assign a higher weight to $X_{kj}$ in the estimation $\hat X_{ij}$. Note that the denominator is just the normalization term so that all the weights sum to 1.

In practice, the above formula raises two issues:

1. The user similarity weights may be positive or negative (i.e., it is possible that $\sum_{k : X_{kj} \ne 0} w_{ik} = 0$), so we should use the absolute weights in the denominator to avoid indeterminate results.
2. We should also, to some extent, account for individual user differences (i.e., some users naturally assign lower ratings than others). We can do so by modeling $X_{ij}$ in the "difference space", the difference between a user’s rating and their mean rating, and then re-scale by adding the target user's mean score.

More specifilly, let $\bar x_i$ be the average of the observed ratings for user $i$, the user-user approach estimation of $X_{ij}$ is

$$\boxed{\hat X_{ij} = \bar x_i + \frac{  \sum_{k : X_{kj} \ne 0} w_{ik}(X_{kj} - \bar x_k)}{  \sum_{k : X_{kj} \ne 0} |w_{ik}|}}$$

What remains is how to compute the weights $w_{ik}$. There are two common options:
1. **Pearson correlation**: let $I_{ik}$ be the set of movies that user $i$ and $k$ both rated. The Pearson correlation weight between user $i$ and $k$ is then
$$\boxed{w_{ik} = \frac{  \sum_{j \in I_{ik}} (X_{ij} - \bar x_i)(X_{kj} - \bar x_j)}{  \sqrt{\sum_{j \in I_{ik}} (X_{ij} - \bar x_i)^2} \sqrt{\sum_{j \in I_{ik}} (X_{kj} - \bar x_k)^2} + \epsilon}}$$
where $\epsilon$ is a small smoothing term for the case when $I_{ik} = \emptyset$, which would otherwise cause the denominator to be 0.
2. **Cosine similarity**: the Cosine similarity, in the context of collaborative filtering, is defined as
$$ \boxed{w_{ik} = \frac{ \sum_{j=1}^n X_{ij} X_{kj}}{ \sqrt{\sum_{j=1}^n X_{ij}^2} \sqrt{\sum_{j=1}^n X_{kj}^2}}}$$

### 2.2 Example
Using the matrix $X$ in Section 1,

$$X = \begin{pmatrix}
    0 & 3 \\
    0 & 1 \\
    2 & 3 \\
\end{pmatrix},$$

let's say we want to estimate $X_{11}$ using the Cosine similarity as weights. We first compute the user-user weight matrix as
$$W = \begin{pmatrix} 1 & 1 & 0.832 \\ 1 & 1 & 0.832 \\ 0.832 & 0.832 & 1 \end{pmatrix}.$$
Then we can compute $\hat X_{11}$ as follows:
$$\hat X_{11} = \bar x_1 + \frac{w_{13} (X_{31} - \bar x_3)}{|w_{13}|} = 3 + \frac{0.832 (2 - 2.5)}{0.832} = 2.5.$$

## 3. Item-item approach

### 3.1 Definition
The item-item approach is analogous to the user-user approach, only now that the roles of users and items are reversed. In other words, to make a prediction about the likely rating for user $i$ and item $j$, we sum over other item ratings that user $i$ has rated, weighted by the similarity between the two items $i$ and $j$:
$$\boxed{\hat X_{ij} = \bar x_j + \frac{ \sum_{k: X_{ik} \ne 0} w_{jk}(X_{ik} - \bar x_k)}{ \sum_{k : X_{ik} \ne 0} |w_{jk}|}} $$
where $\bar x_j$ is the average rating for item $j$ and $w_{jk}$ is the similarity between item $j$ and item $k$ (in terms of Pearson correlation or Cosine similarity again). Note that the $k$ here iterates over the items, not the users.

### 3.2 Example

Using the matrix $X$ in Section 1,

$$X = \begin{pmatrix}
    0 & 3 \\
    0 & 1 \\
    2 & 3 \\
\end{pmatrix},$$

let's say we want to estimate $X_{11}$ using the Cosine similarity as weights. We first compute the item weight matrix as
$$W = \begin{pmatrix} 1 & 0.688 \\ 0.688 & 1 \end{pmatrix}$$

Then we can compute $\hat X_{11}$ as follows:
$$\hat X_{11} = \bar x_1 + \frac{w_{12}(X_{12} - \bar x_2)}{|w_{12}|} = 2 + \frac{0.688(3 - 7/3)}{0.688} = \frac{8}{3} \approx 2.66.$$

## 4. Matrix factorization

### 4.1. Definition
One drawback of the user-user and item-item approaches is that its operations (addition and subtraction) are not suitable for sparse matrix. As you will see from your implementation in Project 1, these methods involve adding the input matrix $X$ with the user means or item means matrix, which results in a dense matrix because every entry in the user/item means matrix is positive. Consequently, the user-rating matrix $X$, despite being sparse in nature, has to be implemented in dense format, which does not scale well with large number of users / items. To address this issue, the matrix factorization approach aims to approximate *the observed entries in $X$* as a product of two lower-rank matrices
$$X \approx \hat X = UV,$$
where $U \in \mathbb{R}^{m \times k}, V \in \mathbb{R}^{k \times n}$ and $k \ll m, n$. Our hope is that, if $\hat X$ can approximate the observed entries in $X$ well, we could use it to predict the missing entries in $X$ as well. Unlike the user-user and item-item approach, however, we do not need to break the sparsity of $X$ to compute $\hat X$. Instead, we only store two lower-rank matrices $U \in \mathbb{R}^{m \times k}$ and $V \in \mathbb{R}^{k \times n}$. Here $k$ is a hyperparameter of our choosing, and since it is typically much smaller than $m$ and $n$, storing $U$ and $V$ in dense format would not pose any issue. Any time we want to get $\hat X$, we just compute $UV$.

More formally, for each user $i = 1, \ldots, m$ we define a set of *user-specific weights* $u_i \in \mathbb{R}^k$ (each $u_i$ is one row in the matrix $U$), and for each item $j = 1, \ldots,n $ we define a set of *item-specific weights* $v_j \in \mathbb{R}^k$ (each $v_j$ is one column in the matrix $V$). For each entry $X_{ij}$, our estimation $\hat X_{ij}$ is
$$\boxed{\hat X_{ij} = u_i^T v_j}$$

Our goal, then, is to learn the weights $u_i$'s and $v_j$'s that minimize the squared loss function

$$\boxed{h(\hat X, X) = \sum_{X_{ij} \ne 0} (u_i^T v_j - X_{ij})^2}$$

In other words, we want the total squared distances between the predicted and actual ratings to be minimized (note that the loss function is only evaluated over the observed ratings).

A commmon method to solve this optimization problem is *alternating least squares*. The underlying motivation here is that it’s difficult to solve the above optimization problem globally over both $U$ and $V$ (in fact this is a non-convex problem, so there is the possibility of local optima). On the other hand, if we **fix all the $u_i$'s terms** (i.e., treat them as constants), then we can find the optimal $v_j$'s terms using the [typical least-squares derivation](http://pillowlab.princeton.edu/teaching/statneuro2018/slides/notes03b_LeastSquaresRegression.pdf):
$$v_j := \left( \sum_{X_{ij} \ne 0} u_i u_i^T \right)^{-1} \left( \sum_{X_{ij} \ne 0} u_i X_{ij} \right), \quad i = 1, \ldots, n$$

where the first bracket is the inverse of a sum of outer products (with dimension $k \times k$), and the second bracket is a sum of vectors (with dimension $k \times 1$).

Similarly, if we **fix all the $v_j$'s terms**, then we can derive $u_i$'s in the same manner:
$$u_i := \left( \sum_{X_{ij} \ne 0} v_j v_j^T \right)^{-1} \left( \sum_{X_{ij} \ne 0} v_j X_{ij} \right), \quad i = 1, \ldots, m$$

This observation inspires a simple approach to solving the problem:

1. Start with some initial random values for $U$ and $V$.
2. Repeat for $N$ times:
    * Update the columns $v_j$'s of $V$ based on the most recent $U$:
    $$\boxed{v_j := \left( \lambda I + \sum_{X_{ij} \ne 0} u_i u_i^T \right)^{-1} \left( \sum_{X_{ij} \ne 0} u_i X_{ij} \right), \quad j = 1, \ldots, n}$$
    * Update the rows $u_i$'s of $U$ based on the most recent $V$:
    $$\boxed{u_i := \left( \lambda I + \sum_{X_{ij} \ne 0} v_j v_j^T \right)^{-1} \left( \sum_{X_{ij} \ne 0} v_j X_{ij} \right), \quad i = 1, \ldots, m}$$

where $\lambda$ is a constant and $I$ is the $k \times k$ identity matrix; in practice, we often add the smoothing term $\lambda I$ to avoid singularity issues with matrix inverses.

Naturally, this approach may not yield the global optimum, but it often works well in practice. As we will see in Project 1, implementation of the alternating least squares algorithm is feasible with sparse matrices, which can considerably reduce runtime and memory usage.

### 4.2 Example 1

Using the matrix $X$ in Section 1,

$$X = \begin{pmatrix}
    0 & 3 \\
    0 & 1 \\
    2 & 3 \\
\end{pmatrix},$$

let's say we pick $k = 1$ (in which case we don't need $\lambda I$) and initialize $U \in \mathbb{R}^{3 \times 1},\ V \in \mathbb{R}^{1 \times 2}$ as 
$$U = \begin{pmatrix} 0.1 \\ 0.2 \\ 0.3 \end{pmatrix}, \quad V = \begin{pmatrix} 0.7 & 0.9 \end{pmatrix}.$$

In the first iteration, we update the columns of $V$ as follows:
\begin{align*}
    v_1 & = \left( \sum_{X_{i1} \ne 0} u_i u_i^T \right)^{-1} \left( \sum_{X_{i1} \ne 0} u_i X_{i1} \right) \\
    & = (u_3 u_3^T)^{-1} (u_3 X_{31}) \\
    & = 0.3^{-2} \cdot (0.3 \cdot 2) = 20/3, \\\\
    v_2 & = \left( \sum_{X_{i2} \ne 0} u_i u_i^T \right)^{-1} \left( \sum_{X_{i2} \ne 0} u_i X_{i2} \right) \\
    & = (u_1u_1^T + u_2u_2^T + u_3u_3^T)^{-1} (u_1X_{12} + u_2X_{22} + u_3X_{32}) \\
    & = (0.1^2 + 0.2^2 + 0.3^2)^{-1} (0.1 \cdot 3 + 0.2 \cdot 1 + 0.3 \cdot 3) = 10.
\end{align*}

Therefore we have new $V = \begin{pmatrix} 20/3 & 10 \end{pmatrix}$. Now, we use this most recent $V$ to update $U$:
\begin{align*}
    u_1 & = \left( \sum_{X_{1j} \ne 0} v_j v_j^T \right)^{-1} \left( \sum_{X_{1j} \ne 0} v_j X_{1j} \right) \\
    & = (v_2 v_2^T)^{-1} (v_2 X_{12}) \\
    & = 10^{-2} \cdot (10 \cdot 3) = 0.3, \\\\
    u_2 & = \left( \sum_{X_{2j} \ne 0} v_j v_j^T \right)^{-1} \left( \sum_{X_{2j} \ne 0} v_j X_{2j} \right) \\
    & = (v_2 v_2^T)^{-1} (v_2 X_{22}) \\
    & = 10^{-2} \cdot (10 \cdot 1) = 0.1, \\\\
    u_3 & = \left( \sum_{X_{3j} \ne 0} v_j v_j^T \right)^{-1} \left( \sum_{X_{3j} \ne 0} v_j X_{3j} \right) \\
    & = (v_1 v_1^T + v_2v_2^T)^{-1} (v_1 X_{31} + v_2 X_{32}) \\
    & = ((20/3)^2 + 10^2)^{-1} (20/3 \cdot 2 + 10 \cdot 3) = 0.3.
\end{align*}

Therefore we have new $U = \begin{pmatrix} 0.3 \\ 0.1 \\ 0.3 \end{pmatrix}$. If we compute $\hat X$ using the new $U$ and $V$, we would have
$$\hat X = UV = \begin{pmatrix} 0.3 \\ 0.1 \\ 0.3 \end{pmatrix} \begin{pmatrix} 20/3 & 10 \end{pmatrix} = \begin{pmatrix} 2 & 3 \\ 0.66 & 1 \\ 2 & 3 \end{pmatrix}$$
and would then infer the missing values in $X$ as: $\hat X_{11} = 2$ and $\hat X_{21} = 0.66$.

In this very simple case, we see that the squared loss function $h(\hat X, X) = \sum_{X_{ij} \ne 0} (u_i^T v_j - X_{ij})^2$ is already 0, but typically it will require more iterations to get closer to 0.

### 4.3. Example 2
Using the matrix $X$ as follows,

$$X = \begin{pmatrix}
    0 & 0 & 1 \\
    3 & 1 & 0 \\
    0 & 0 & 3 \\
    4 & 0 & 0
\end{pmatrix},$$

let's say we pick $k = 2, \lambda = 10^{-1}$ and initialize $U \in \mathbb{R}^{4 \times 2},\ V \in \mathbb{R}^{2 \times 3}$ as 
$$U = \begin{pmatrix} 0.1 & 0.5 \\ 0.2 & 0.6 \\ 0.3 & 0.7 \\ 0.4 & 0.8 \end{pmatrix}, \quad V = \begin{pmatrix} 0.15 & 0.25 & 0.35 \\ 0.45 & 0.55 & 0.65 \end{pmatrix}.$$

In the first iteration, we update the columns of $V$ as follows:
\begin{align*}
    v_1 & = \left( \lambda I + \sum_{X_{i1} \ne 0} u_i u_i^T \right)^{-1} \left( \sum_{X_{i1} \ne 0} u_i X_{i1} \right) \\
    & = (10^{-1} I + u_2 u_2^T + u_4 u_4^T)^{-1} (u_2 X_{21} + u_4 X_{41}) \\
    & = \left[ 10^{-1} \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} + \begin{pmatrix} 0.2\\0.6 \end{pmatrix} \begin{pmatrix} 0.2 & 0.6 \end{pmatrix} + \begin{pmatrix} 0.4\\0.8 \end{pmatrix} \begin{pmatrix} 0.4 & 0.8 \end{pmatrix} \right]^{-1} \left[ 3\begin{pmatrix} 0.2\\0.6 \end{pmatrix} + 4\begin{pmatrix} 0.4\\0.8 \end{pmatrix} \right] \\
    & = \begin{pmatrix} 1.613 \\ 3.900 \end{pmatrix}, \\\\
    v_2 & = \left( \lambda I + \sum_{X_{i2} \ne 0} u_i u_i^T \right)^{-1} \left( \sum_{X_{i2} \ne 0} u_i X_{i2} \right) \\
    & = (10^{-1} I + u_2 u_2^T)^{-1} (u_2 X_{22}) \\
    & = \left[10^{-1} \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} + \begin{pmatrix} 0.2\\0.6 \end{pmatrix} \begin{pmatrix} 0.2 & 0.6 \end{pmatrix} \right]^{-1} \begin{pmatrix} 0.2\\0.6 \end{pmatrix} \\
    & = \begin{pmatrix} 0.400 \\ 1.200 \end{pmatrix}, \\\\
    v_3 & = \left( \lambda I + \sum_{X_{i3} \ne 0} u_i u_i^T \right)^{-1} \left( \sum_{X_{i3} \ne 0} u_i X_{i3} \right) \\
    & = (10^{-1} I + u_1 u_1^T + u_3 u_3^T)^{-1} (u_1 X_{13} + u_3 X_{33}) \\
    & = \left[ 10^{-1} \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} + \begin{pmatrix} 0.1\\0.5 \end{pmatrix} \begin{pmatrix} 0.1 & 0.5 \end{pmatrix} + \begin{pmatrix} 0.3\\0.7 \end{pmatrix} \begin{pmatrix} 0.3 & 0.7 \end{pmatrix} \right]^{-1} \left[ \begin{pmatrix} 0.1\\0.5 \end{pmatrix} + 3\begin{pmatrix} 0.3\\0.7 \end{pmatrix} \right] \\
    & = \begin{pmatrix} 1.633 \\ 2.590 \end{pmatrix}. \\\\ 
\end{align*}

Therefore we have the new $V$ as
$$ V = \begin{pmatrix} 1.613 & 0.400 & 1.633 \\ 3.900 & 1.200 & 2.590 \end{pmatrix}.$$

Using this most recent $V$, we can similarly update $U$ to obtain the new $U$ as

$$U = \begin{pmatrix}
    0.172 & 0.273 \\
    0.197 & 0.691 \\
    0.517 & 0.820 \\
    0.360 & 0.871 \\
\end{pmatrix},$$

which in turn yields the predicted rating matrix

$$\hat X = UV = \begin{pmatrix} 
    1.344 & 0.397 & 0.989 \\
    3.011 & 0.907 & 2.110 \\
    4.032 & 1.191 & 2.968 \\
    3.978 & 1.189 & 2.844
\end{pmatrix}.$$

Comparing this to the original matrix

$$X = \begin{pmatrix}
    0 & 0 & 1 \\
    3 & 1 & 0 \\
    0 & 0 & 3 \\
    4 & 0 & 0
\end{pmatrix},$$

we see that the observed (non-zero) entries in $X$ are approximated fairly well. The squared loss value $h(\hat X, X) = \sum_{X_{ij} \ne 0} (u_i^T v_j - X_{ij})^2$ in this case is only 0.01, and will be closer to 0 with more iterations.